In [ ]:
# Import libraries
import torch
import pandas as pd
import numpy as np

import sys
import os

sys.path.append(os.path.join(os.path.abspath(''), '../pyno/lib'))

from autoencoder import Autoencoder
from auto_threshold_re import AutoThresholdRe
from cnn_autoencoder import CnnAutoencoder

In [ ]:
# Parameters
training_file = "/home/ralampay/workspace/pyno/data/ucsd-cnn-training.csv"
test_dir = "/home/ralampay/Downloads/UCSD_Anomaly_Dataset.vip2/UCSDped1/Test/Test001/"
model_file = "/home/ralampay/workspace/pyno/models/ucsd-cnn.pth"
layers = []
epochs = 100
lr = 0.001
batch_size = 5
device = "cuda"
gpu_index = 0
chunk_size = 100

In [ ]:
# Train unsupervised autoencoder
if device != "cpu":
    device = "{}:{}".format(device, gpu_index)

initial_autoencoder = Autoencoder(layers=layers, device=device)

data = pd.DataFrame()

for i, chunk in enumerate(pd.read_csv(training_file, header=None, chunksize=chunk_size)):
    data = data.append(chunk)
    
x = torch.tensor(data.values).float().to(device)

initial_autoencoder.fit(x, epochs=epochs, lr=lr, batch_size=batch_size)

x_hat = initial_autoencoder.forward(x)

err = (x_hat - x).pow(2).sum(dim=1).sqrt()

err_values = err.detach().cpu().numpy()
bool_array = err_values >= initial_autoencoder.reconstruction_threshold.detach().cpu().numpy()

predictions = np.array([-1 if elem else 1 for elem in bool_array])

data_values = data.values

filtered_data = pd.DataFrame()

for i in range(len(predictions)):
    if predictions[i] != -1:
        filtered_data = filtered_data.append([data_values[i]])
        
print("Reduced data from {} to {}".format(len(data), len(filtered_data)))

autoencoder = Autoencoder(layers=layers, device=device)

filtered_data_values = filtered_data.values

x = torch.tensor(filtered_data_values).float().to(device)

autoencoder.fit(x, epochs=epochs, lr=lr, batch_size=batch_size)

autothreshold_ops = AutoThresholdRe(x, autoencoder)
autothreshold_ops.execute()

anomaly_threshold = autothreshold_ops.optimal_threshold

In [ ]:
# Quantize test frames

scale
channel_maps
padding
kernel_size
num_channels
img_width
img_height

In [ ]:
# Detect anomalies